# Baseline RAG テスト

こちらを参考にBaseline RAGを構築TEST：

https://note.com/noa813/n/n6065209e7bf0

### ライブラリーのインストール
!pip install langchain openai chromadb tiktoken

### OpenAIのAPIキーを環境変数に格納

In [1]:
import openai
import os
import getpass

# !初回のみ! openaiのAPIキーを設定

# OpenAI APIのアカウントを作成、
# https://platform.openai.com/account/api-keys
# からAPIキーを取得して環境変数に設定

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

openai.api_key = os.getenv("OPENAI_API_KEY")

### LLMが参照するテキストを用意

In [2]:
file_path = './data/manufacturing_company_dummy.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

text


'名前： 山田太郎\n所属部門： 製造\n所属部署： 生産統括\n役職： 部長\n経歴詳細：\n本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持つ。佐藤健太とは10年前にB工程ラインで共にライン管理に従事し、B工程に関する知識も豊富である。今まで関わったプロジェクトとしては以下がある：\nA工程ランニングコスト削減プロジェクト\nB工程製造設備オーバーホールプロジェクト\n新卒人材採用活動\n*****\n名前： 佐藤健太\n所属部門： 製造\n所属部署： 生産統括\n役職： 課長\n経歴詳細：\n入社15年目で、B工程の生産ライン管理を担当。山田太郎と協働し、B工程製造設備オーバーホールプロジェクトを成功に導いた。これまで関わったプロジェクトは以下の通り：\nB工程製造設備オーバーホールプロジェクト\nC工程生産性向上プロジェクト\n安全管理体制強化プロジェクト\n*****\n名前： 鈴木花子\n所属部門： 品質管理\n所属部署： 品質保証\n役職： マネージャー\n経歴詳細：\n品質管理のエキスパートとして10年以上の経験を持つ。田中一郎と共に品質管理システム改善プロジェクトをリードし、製品の品質向上に貢献。参加したプロジェクト：\n品質管理システム改善プロジェクト\n品質認証取得プロジェクト\n環境負荷低減プロジェクト\n*****\n名前： 田中一郎\n所属部門： 品質管理\n所属部署： 品質保証\n役職： スペシャリスト\n経歴詳細：\n品質保証のスペシャリストとして、鈴木花子と協力し品質管理システム改善プロジェクトを推進。以下のプロジェクトに関わった：\n品質管理システム改善プロジェクト\n業務プロセス改善プロジェクト\n安全管理体制強化プロジェクト\n*****\n名前： 高橋直樹\n所属部門： 製造\n所属部署： 製造技術\n役職： 技師\n経歴詳細：\n製造技術部門で新たな生産技術の導入に尽力。佐々木優と共に生産ライン自動化導入プロジェクトを成功させ、生産効率を大幅に向上させた。関与したプロジェクト：\n生産ライン自動化導入プロジェクト\nD工程作業手順書改訂プロジェクト\n環境負荷低減プロジェクト\n*****\n名前： 佐々木優\n所属部門： 製造\n所属部署： 製造技術\n役職： エンジニア\n経歴詳細：\n

### テキストを分割

In [3]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="*****",
    chunk_size=100, 
    chunk_overlap=0
    )

docs = text_splitter.split_text(text)

len(docs)

Created a chunk of size 204, which is longer than the specified 100
Created a chunk of size 176, which is longer than the specified 100
Created a chunk of size 170, which is longer than the specified 100
Created a chunk of size 157, which is longer than the specified 100
Created a chunk of size 166, which is longer than the specified 100
Created a chunk of size 162, which is longer than the specified 100
Created a chunk of size 154, which is longer than the specified 100
Created a chunk of size 167, which is longer than the specified 100
Created a chunk of size 156, which is longer than the specified 100
Created a chunk of size 156, which is longer than the specified 100
Created a chunk of size 143, which is longer than the specified 100
Created a chunk of size 135, which is longer than the specified 100
Created a chunk of size 131, which is longer than the specified 100
Created a chunk of size 155, which is longer than the specified 100
Created a chunk of size 150, which is longer tha

60

### OpenAIのEmbeddingのインスタンス化

* チャンクのベクトル化を行っている
* 追加インストールが必要：pip install -U langchain-community
* !Warning有り! 

LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()



C:\Users\user\AppData\Local\Temp\ipykernel_18660\3921495447.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


### VectorStore作成

In [5]:
from langchain.vectorstores import Chroma
db = Chroma.from_texts(docs, embeddings)
retriever = db.as_retriever()

### 質問（Query）を用意

質問（query）がベクトル化され、
queryと類似度が近いベクトルを持つ
テキストをVectorStoreから検索します。
そして、
queryに関連する文書が
context_docsへ格納されます

In [9]:
# query = "人的資本の強化のために取り組んでいることは何でしょうか？"
query = "山田太郎と一緒にプロジェクトに従事したことがある社員を全員挙げてください"

context_docs = retriever.get_relevant_documents(query)
context_docs

[Document(metadata={}, page_content='名前： 上田健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n原田涼と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 Document(metadata={}, page_content='名前： 田村亮太\n所属部門： IT\n所属部署： システム開発\n役職： エンジニア\n経歴詳細：\n吉田健とITシステム統合プロジェクトに参加。業務プロセス改善プロジェクトでも技術的支援を提供。参加したプロジェクト：\nITシステム統合プロジェクト\n業務プロセス改善プロジェクト\n生産ライン自動化導入プロジェクト'),
 Document(metadata={}, page_content='名前： 吉田健\n所属部門： IT\n所属部署： システム開発\n役職： エンジニア\n経歴詳細：\n石田光とITシステム統合プロジェクトに参加。業務プロセス改善プロジェクトでも技術的支援を提供。参加したプロジェクト：\nITシステム統合プロジェクト\n業務プロセス改善プロジェクト\n生産ライン自動化導入プロジェクト'),
 Document(metadata={}, page_content='名前： 藤田翔太\n所属部門： 製造\n所属部署： 生産管理\n役職： エンジニア\n経歴詳細：\n橋本健太郎と在庫管理効率化プロジェクトに参加。C工程生産性向上プロジェクトでも貢献。参加したプロジェクト：\n在庫管理効率化プロジェクト\nC工程生産性向上プロジェクト\n業務プロセス改善プロジェクト')]

### *** LLMに質問を投げ、RAGを使って回答

In [11]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

result = qa_chain.run(query)
print(result)

田村亮太、吉田健、藤田翔太


### ChatGPT　モデル間の回答比較

##### ダミーデータ(製造業企業の社員情報) で検証
**Query #1:「山田太郎はどのような人物ですか？」**

*retriever結果(どのチャンクが回答根拠として選ばれたか？)


[Document(metadata={}, page_content='名前： 山田太郎\n所属部門： 製造\n所属部署： 生産統括\n役職： 部長\n経歴詳細：\n本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持つ。佐藤健太とは10年前にB工程ラインで共にライン管理に従事し、B工程に関する知識も豊富である。今まで関わったプロジェクトとしては以下がある：\nA工程ランニングコスト削減プロジェクト\nB工程製造設備オーバーホールプロジェクト\n新卒人材採用活動'),
 
 
 Document(metadata={}, page_content='名前： 上田健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n原田涼と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 谷口健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n上田健太と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 橋本健太郎\n所属部門： 製造\n所属部署： 生産管理\n役職： 課長\n経歴詳細：\n生産ラインの効率化に注力。木村拓也と在庫管理効率化プロジェクトで協力。参加したプロジェクト：\n在庫管理効率化プロジェクト\nコスト削減アイデア募集活動\n業務プロセス改善プロジェクト')]



**LLMの回答**

* turbo-3.5
「山田太郎は製造部門に所属し、生産統括部署の部長です。20年以上の経験を持ち、A工程の管理・改善に関わってきました。10年前にはB工程ラインでライン管理に従事し、B工程にも豊富な知識を持っています。過去に関わったプロジェクトにはA工程ランニングコスト削減プロジェクト、B工程製造設備オーバーホールプロジェクト、新卒人材採用活動などがあります。」

* 4o

「山田太郎は、製造部門の生産統括部署に所属する部長です。本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持ち、B工程に関する知識も豊富です。彼はA工程ランニングコスト削減プロジェクトやB工程製造設備オーバーホールプロジェクト、新卒人材採用活動などに関わってきました。また、佐藤健太とは10年前にB工程ラインで共にライン管理に従事した経験があります。」





　
　
　
　
　



**Query #2:「山田太郎と一緒にプロジェクトに従事したことがある社員を全員挙げてください」**

*retriever結果(どのチャンクが回答根拠として選ばれたか？)



**LLMの回答**

* turbo-3.5
「田村亮太、吉田健、藤田翔太」
-> <font color="red">不正解</font>

* 4o
「<font color="red">申し訳ありませんが、提供された情報の中に山田太郎という名前の社員は含まれていません。そのため、山田太郎と一緒にプロジェクトに従事したことがある社員についてはわかりません。</font>
」

